# PyCity Schools Analysis

Two unique observations:

-- While charter schools have higher average scores than district schools in both math and reading, district schools outperform charter schools in % of students who pass either and both courses. See "Scores by School Type" Table

--Interestingly, grades and passing rates do not seem to increase as per-student spending rises. While you would expect more money spent per student would correlate to student success, no such correlation is shown in this data set.

In [562]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [563]:
# Calculate the total number of unique schools
unique_schools = school_data_complete['school_name'].unique()
school_count = len(unique_schools)
school_count

15

In [564]:
# Calculate the total number of students
student_count = len(school_data_complete['Student ID'])
student_count

39170

In [565]:
# Calculate the total budget
unique_budget = school_data_complete['budget'].unique()
total_budget = sum(unique_budget)
total_budget

24649428

In [566]:
# Calculate the average (mean) math score
average_math_score = sum(school_data_complete['math_score']) / len(school_data_complete['math_score'])
average_math_score

78.98537145774827

In [567]:
# Calculate the average (mean) reading score
average_reading_score = sum(school_data_complete['reading_score']) / len(school_data_complete['reading_score'])
average_reading_score

81.87784018381414

In [568]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [569]:
# Calculate the percentage of students who passed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [570]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [571]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary_data = {
    "Total Schools": school_count,
    "Total Students": student_count,
    "Total Budget": total_budget,
    "Average Math Score": average_math_score,
    "Average Reading Score": average_reading_score,
    "% Passing Math": passing_math_percentage,
    "% Passing Reading": passing_reading_percentage,
    "% Passing Both": overall_passing_rate
}

district_summary = pd.DataFrame(district_summary_data, index=[0])

# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Both
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [572]:
# Use the code provided to select all of the school types
school_types = school_data_complete[["school_name", 'type']]
school_types = school_types.drop_duplicates(subset='school_name')
school_types

,school_name,type
0,Huang High School,District
2917,Figueroa High School,District
5866,Shelton High School,Charter
7627,Hernandez High School,District
12262,Griffin High School,Charter
13730,Wilson High School,Charter
16013,Cabrera High School,Charter
17871,Bailey High School,District
22847,Holden High School,Charter
23274,Pena High School,Charter


In [573]:
# Calculate the total student count per school
per_school_counts = school_data_complete["school_name"].value_counts()
per_school_counts

school_name
Bailey High School       4976
Johnson High School      4761
Hernandez High School    4635
Rodriguez High School    3999
Figueroa High School     2949
Huang High School        2917
Ford High School         2739
Wilson High School       2283
Cabrera High School      1858
Wright High School       1800
Shelton High School      1761
Thomas High School       1635
Griffin High School      1468
Pena High School          962
Holden High School        427
Name: count, dtype: int64

In [574]:
# Calculate the total school budget and per capita spending per school
#School Budgets
budget_repeating = school_data_complete[["school_name", "budget"]]
per_school_budget = budget_repeating.drop_duplicates()
per_school_budget

,school_name,budget
0,Huang High School,1910635
2917,Figueroa High School,1884411
5866,Shelton High School,1056600
7627,Hernandez High School,3022020
12262,Griffin High School,917500
13730,Wilson High School,1319574
16013,Cabrera High School,1081356
17871,Bailey High School,3124928
22847,Holden High School,248087
23274,Pena High School,585858


In [575]:
#Create a Dataset with Both Budget and Counts
budget_per_capita = pd.merge(per_school_budget, per_school_counts, on='school_name')
budget_per_capita["Budget Per Capita"] = budget_per_capita["budget"] / budget_per_capita["count"]
budget_per_capita.rename(columns={"count" : "Number of Students"}, inplace=True)
budget_per_capita

,school_name,budget,Number of Students,Budget Per Capita
0,Huang High School,1910635,2917,655.0
1,Figueroa High School,1884411,2949,639.0
2,Shelton High School,1056600,1761,600.0
3,Hernandez High School,3022020,4635,652.0
4,Griffin High School,917500,1468,625.0
5,Wilson High School,1319574,2283,578.0
6,Cabrera High School,1081356,1858,582.0
7,Bailey High School,3124928,4976,628.0
8,Holden High School,248087,427,581.0
9,Pena High School,585858,962,609.0


In [576]:
# Calculate the average test scores per school
test_scores_per_school = school_data_complete[["school_name", "math_score", "reading_score"]]

test_scores_by_school = test_scores_per_school.groupby('school_name')

means_by_school = test_scores_by_school.mean()
means_by_school.rename(columns={"math_score": "Average Math Score"}, inplace= True)
means_by_school.rename(columns={"reading_score": "Average Reading Score"}, inplace= True)

means_by_school

,Average Math Score,Average Reading Score
school_name,,
Bailey High School,77.048432,81.033963
Cabrera High School,83.061895,83.975780
Figueroa High School,76.711767,81.158020
Ford High School,77.102592,80.746258
Griffin High School,83.351499,83.816757
Hernandez High School,77.289752,80.934412
Holden High School,83.803279,83.814988
Huang High School,76.629414,81.182722
Johnson High School,77.072464,80.966394


In [598]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math = school_data_complete[school_data_complete['math_score'] >= 70]

school_students_passing_math = students_passing_math.groupby(["school_name"]).size()

school_students_passing_math

school_name
Bailey High School       3318
Cabrera High School      1749
Figueroa High School     1946
Ford High School         1871
Griffin High School      1371
Hernandez High School    3094
Holden High School        395
Huang High School        1916
Johnson High School      3145
Pena High School          910
Rodriguez High School    2654
Shelton High School      1653
Thomas High School       1525
Wilson High School       2143
Wright High School       1680
dtype: int64

In [599]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = school_data_complete[school_data_complete['reading_score'] >= 70]
school_students_passing_reading = students_passing_reading.groupby(["school_name"]).size()

school_students_passing_reading

school_name
Bailey High School       4077
Cabrera High School      1803
Figueroa High School     2381
Ford High School         2172
Griffin High School      1426
Hernandez High School    3748
Holden High School        411
Huang High School        2372
Johnson High School      3867
Pena High School          923
Rodriguez High School    3208
Shelton High School      1688
Thomas High School       1591
Wilson High School       2204
Wright High School       1739
dtype: int64

In [600]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()

school_students_passing_math_and_reading

school_name
Bailey High School       2719
Cabrera High School      1697
Figueroa High School     1569
Ford High School         1487
Griffin High School      1330
Hernandez High School    2481
Holden High School        381
Huang High School        1561
Johnson High School      2549
Pena High School          871
Rodriguez High School    2119
Shelton High School      1583
Thomas High School       1487
Wilson High School       2068
Wright High School       1626
dtype: int64

In [580]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_students_passing_math / student_count * 100
per_school_passing_reading = school_students_passing_reading / student_count * 100
overall_passing_rate = school_students_passing_math_and_reading / student_count * 100

overall_passing_rate

school_name
Bailey High School       6.941537
Cabrera High School      4.332397
Figueroa High School     4.005617
Ford High School         3.796273
Griffin High School      3.395456
Hernandez High School    6.333929
Holden High School       0.972683
Huang High School        3.985193
Johnson High School      6.507531
Pena High School         2.223641
Rodriguez High School    5.409752
Shelton High School      4.041358
Thomas High School       3.796273
Wilson High School       5.279551
Wright High School       4.151136
dtype: float64

In [602]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.merge(budget_per_capita, means_by_school, on= "school_name")
per_school_summary = pd.merge(per_school_summary, school_types, on="school_name")
per_school_summary.rename(columns={"budget" : "Total School Budget (In Dollars)"}, inplace= True)
per_school_summary.rename(columns={"Budget Per Capita" : "Per Student Budget (In Dollars)"}, inplace= True)
per_school_summary["% Passing Math"] = per_school_passing_math.values
per_school_summary["% Passing Reading"] = per_school_passing_reading.values
per_school_summary["% Passing Overall"] = overall_passing_rate.values
per_school_summary.rename(columns= {"school_name" : "School Name"}, inplace=True)
per_school_summary.rename(columns= {"type" : "School Type"}, inplace=True)

# Display the DataFrame
per_school_summary

,School Name,Total School Budget (In Dollars),Number of Students,Per Student Budget (In Dollars),Average Math Score,Average Reading Score,School Type,% Passing Math,% Passing Reading,% Passing Overall
0,Huang High School,1910635,2917,655.0,76.629414,81.182722,District,8.470768,10.408476,6.941537
1,Figueroa High School,1884411,2949,639.0,76.711767,81.158020,District,4.465152,4.603013,4.332397
2,Shelton High School,1056600,1761,600.0,83.359455,83.725724,Charter,4.968088,6.078632,4.005617
3,Hernandez High School,3022020,4635,652.0,77.289752,80.934412,District,4.776615,5.545060,3.796273
4,Griffin High School,917500,1468,625.0,83.351499,83.816757,Charter,3.500128,3.640541,3.395456
5,Wilson High School,1319574,2283,578.0,83.274201,83.989488,Charter,7.898902,9.568547,6.333929
6,Cabrera High School,1081356,1858,582.0,83.061895,83.975780,Charter,1.008425,1.049272,0.972683
7,Bailey High School,3124928,4976,628.0,77.048432,81.033963,District,4.891499,6.055655,3.985193
8,Holden High School,248087,427,581.0,83.803279,83.814988,Charter,8.029104,9.872351,6.507531
9,Pena High School,585858,962,609.0,83.839917,84.044699,Charter,2.323207,2.356395,2.223641


## Highest-Performing Schools (by % Overall Passing)

In [582]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(by='% Passing Overall', ascending= False)
top_schools.head(5)

,School Name,Total School Budget (In Dollars),Number of Students,Per Student Budget (In Dollars),Average Math Score,Average Reading Score,School Type,% Passing Math,% Passing Reading,% Passing Overall
0,Huang High School,1910635,2917,655.0,76.629414,81.182722,District,8.470768,10.408476,6.941537
8,Holden High School,248087,427,581.0,83.803279,83.814988,Charter,8.029104,9.872351,6.507531
5,Wilson High School,1319574,2283,578.0,83.274201,83.989488,Charter,7.898902,9.568547,6.333929
10,Wright High School,1049400,1800,583.0,83.682222,83.955000,Charter,6.775594,8.189941,5.409752
13,Ford High School,1763916,2739,644.0,77.102592,80.746258,District,5.471024,5.626755,5.279551


## Bottom Performing Schools (By % Overall Passing)

In [583]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(by='% Passing Overall', ascending= True)
bottom_schools.head(5)

,School Name,Total School Budget (In Dollars),Number of Students,Per Student Budget (In Dollars),Average Math Score,Average Reading Score,School Type,% Passing Math,% Passing Reading,% Passing Overall
6,Cabrera High School,1081356,1858,582.0,83.061895,83.975780,Charter,1.008425,1.049272,0.972683
9,Pena High School,585858,962,609.0,83.839917,84.044699,Charter,2.323207,2.356395,2.223641
4,Griffin High School,917500,1468,625.0,83.351499,83.816757,Charter,3.500128,3.640541,3.395456
3,Hernandez High School,3022020,4635,652.0,77.289752,80.934412,District,4.776615,5.545060,3.796273
12,Johnson High School,3094650,4761,650.0,77.072464,80.966394,District,3.893286,4.061782,3.796273


## Math Scores by Grade

In [584]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_graders_grouped = ninth_graders.groupby(["school_name"])
ninth_grade_math_scores = ninth_graders_grouped["math_score"].mean()

tenth_graders_grouped = tenth_graders.groupby(["school_name"])
tenth_grader_math_scores = tenth_graders_grouped["math_score"].mean()

eleventh_graders_grouped = eleventh_graders.groupby(["school_name"])
eleventh_grader_math_scores = eleventh_graders_grouped["math_score"].mean()

twelfth_graders_grouped = twelfth_graders.groupby(["school_name"])
twelfth_grader_math_scores = twelfth_graders_grouped["math_score"].mean()


# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade= pd.merge(ninth_grade_math_scores, tenth_grader_math_scores, on="school_name")
math_scores_by_grade.rename(columns={"math_score_x" : "9th Grade"}, inplace= True)
math_scores_by_grade.rename(columns={"math_score_y" : "10th Grade"}, inplace= True)
math_scores_by_grade["11th Grade"] = eleventh_grader_math_scores.values
math_scores_by_grade["12th Grade"] = twelfth_grader_math_scores.values

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [585]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = ninth_graders_grouped["reading_score"].mean()
tenth_grader_reading_scores = tenth_graders_grouped["reading_score"].mean()
eleventh_grader_reading_scores = eleventh_graders_grouped["reading_score"].mean()
twelfth_grader_reading_scores = twelfth_graders_grouped["reading_score"].mean()

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.merge(ninth_grade_reading_scores, tenth_grader_reading_scores, on="school_name")
reading_scores_by_grade.rename(columns={"reading_score_x" : "9th Grade"}, inplace= True)
reading_scores_by_grade.rename(columns={"reading_score_y" : "10th Grade"}, inplace= True)
reading_scores_by_grade["11th Grade"] = eleventh_grader_reading_scores.values
reading_scores_by_grade["12th Grade"] = twelfth_grader_reading_scores.values

# Minor data wrangling
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [586]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [587]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [588]:
# Use `pd.cut` to categorize spending based on the bins.
#school_spending_df['Per Student Budget'] = pd.to_numeric(school_spending_df['Per Student Budget'])
school_spending_df['Spending Ranges (Per Student)'] = pd.cut(school_spending_df['Per Student Budget (In Dollars)'], bins = spending_bins, labels = labels)

In [589]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Overall"].mean()

In [590]:
# Assemble into DataFrame
spending_summary = pd.merge(spending_math_scores, spending_reading_scores, on = "Spending Ranges (Per Student)")
spending_summary = pd.merge(spending_summary, spending_passing_math, on = "Spending Ranges (Per Student)")
spending_summary = pd.merge(spending_summary, spending_passing_reading, on = "Spending Ranges (Per Student)")
spending_summary = pd.merge(spending_summary, overall_passing_spending, on = "Spending Ranges (Per Student)")

# Display results
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,5.928006,7.170028,4.805974
$585-630,81.899826,83.155286,3.920730,4.532806,3.402476
$630-645,78.518855,81.624473,4.611310,4.744703,4.451111
$645-680,76.997210,81.027843,5.713556,6.671773,4.844694


## Scores by School Size

In [591]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [592]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_summary['Number of Students'], bins = size_bins, labels = labels)
per_school_summary

,School Name,Total School Budget (In Dollars),Number of Students,Per Student Budget (In Dollars),Average Math Score,Average Reading Score,School Type,% Passing Math,% Passing Reading,% Passing Overall,School Size
0,Huang High School,1910635,2917,655.0,76.629414,81.182722,District,8.470768,10.408476,6.941537,Large (2000-5000)
1,Figueroa High School,1884411,2949,639.0,76.711767,81.158020,District,4.465152,4.603013,4.332397,Large (2000-5000)
2,Shelton High School,1056600,1761,600.0,83.359455,83.725724,Charter,4.968088,6.078632,4.005617,Medium (1000-2000)
3,Hernandez High School,3022020,4635,652.0,77.289752,80.934412,District,4.776615,5.545060,3.796273,Large (2000-5000)
4,Griffin High School,917500,1468,625.0,83.351499,83.816757,Charter,3.500128,3.640541,3.395456,Medium (1000-2000)
5,Wilson High School,1319574,2283,578.0,83.274201,83.989488,Charter,7.898902,9.568547,6.333929,Large (2000-5000)
6,Cabrera High School,1081356,1858,582.0,83.061895,83.975780,Charter,1.008425,1.049272,0.972683,Medium (1000-2000)
7,Bailey High School,3124928,4976,628.0,77.048432,81.033963,District,4.891499,6.055655,3.985193,Large (2000-5000)
8,Holden High School,248087,427,581.0,83.803279,83.814988,Charter,8.029104,9.872351,6.507531,Small (<1000)
9,Pena High School,585858,962,609.0,83.839917,84.044699,Charter,2.323207,2.356395,2.223641,Small (<1000)


In [593]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Passing Overall"].mean()

In [594]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.merge(size_math_scores, size_reading_scores, on = "School Size")
size_summary = pd.merge(size_summary, size_passing_math, on= "School Size")
size_summary = pd.merge(size_summary, size_passing_reading, on= "School Size")
size_summary = pd.merge(size_summary, size_overall_passing, on= "School Size")

# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
Small (<1000),83.821598,83.929843,5.176155,6.114373,4.365586
Medium (1000-2000),83.374684,83.864438,4.108246,4.679602,3.586929
Large (2000-5000),77.746417,81.344493,5.510914,6.272339,4.813314


## Scores by School Type

In [595]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Passing Overall"].mean()

In [596]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.merge(average_math_score_by_type, average_reading_score_by_type, on = "School Type")
type_summary = pd.merge(type_summary, average_percent_passing_math_by_type, on = "School Type")
type_summary = pd.merge(type_summary, average_percent_passing_reading_by_type, on = "School Type")
type_summary = pd.merge(type_summary, average_percent_overall_passing_by_type, on = "School Type")



# Display results
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Type,,,,,
Charter,83.473852,83.896421,4.849055,5.649413,4.124968
District,76.956733,80.966636,5.169773,5.801452,4.596083
